# Analyse des Correspondances Multiples (ACM)
### Dataset Titanic Spaceship — Kaggle

---

## C'est quoi une ACM ?

Imaginez que vous avez un tableau avec des centaines de passagers et plusieurs questions sur chacun d'eux : *D'où vient-il ? Est-il en cryosommeil ? Quelle est sa destination ?*

L'ACM est une méthode qui permet de **résumer visuellement** toutes ces informations en un seul graphique, appelé **plan factoriel**.

Sur ce graphique :
- Les **modalités proches** (ex. Europa et CabinDeck B) apparaissent souvent ensemble dans les données
- Les **modalités éloignées** sont rarement associées
- Plus une modalité est **loin du centre**, plus elle est rare et caractéristique

---

> **Ce dont vous avez besoin :** `pandas`, `numpy`, `matplotlib`, `scikit-learn`, `scipy`, `openpyxl`

> Installez-les avec : `pip install pandas numpy matplotlib scikit-learn scipy openpyxl`

## Étape 1 — Charger les outils nécessaires

Avant de commencer, on charge les librairies Python dont on a besoin.
Pas besoin de tout comprendre ici — exécutez juste cette cellule !

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from sklearn.preprocessing import OneHotEncoder
from scipy.linalg import svd
import warnings
warnings.filterwarnings('ignore')

print('Tout est prêt !')

## Étape 2 — Charger et explorer les données

On charge notre fichier et on choisit les variables qui nous intéressent.

Pour cette ACM, on utilise **7 variables catégorielles** (c'est-à-dire des variables avec des catégories, pas des chiffres) :

| Variable | Ce qu'elle représente |
|----------|----------------------|
| HomePlanet | La planète d'origine du passager |
| CryoSleep | Est-il en cryosommeil ? (0 = non, 1 = oui) |
| Destination | La destination du voyage |
| VIP | A-t-il un statut VIP ? (0 = non, 1 = oui) |
| Transported | A-t-il été transporté ? (variable cible) |
| CabinDeck | Le pont de sa cabine (A, B, C...) |
| CabinSide | Côté de la cabine (P = Port, S = Starboard) |

In [ ]:
# Chargement du fichier — adaptez le chemin si nécessaire
df = pd.read_excel('train_data_cleaned.xlsx')

print(f'Le dataset contient {df.shape[0]} passagers et {df.shape[1]} colonnes.')
print()
df.head()

In [ ]:
# On sélectionne uniquement nos 7 variables catégorielles
cat_vars = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP',
            'Transported', 'CabinDeck', 'CabinSide']

df_mca = df[cat_vars].copy()
for col in cat_vars:
    df_mca[col] = df_mca[col].astype(str)

# On supprime les lignes incomplètes
df_mca = df_mca.dropna()

print(f'Nombre de passagers utilisés pour l analyse : {len(df_mca)}')
print()

# Affichage des catégories disponibles pour chaque variable
print('Catégories disponibles par variable :')
for col in cat_vars:
    cats = sorted(df_mca[col].unique().tolist())
    print(f'  {col:<15} -> {cats}')

## Étape 3 — Calculer l'ACM

C'est ici que la 'magie' mathématique se passe.

En résumé, l'ACM va :
1. Transformer nos variables en un grand tableau de 0 et de 1
2. Chercher les **directions principales** (les axes) qui résument le mieux les données
3. Calculer les **coordonnées** de chaque catégorie sur ces axes

Ne vous inquiétez pas des détails — exécutez juste la cellule !

In [ ]:
# --- Transformation des données ---
# On transforme chaque catégorie en colonne binaire (0 ou 1)
enc = OneHotEncoder(sparse_output=False)
Z   = enc.fit_transform(df_mca)
feature_names = enc.get_feature_names_out(cat_vars)

n = len(df_mca)       # nombre de passagers
K = len(cat_vars)     # nombre de variables
Q = Z.shape[1]        # nombre total de catégories

# --- Calcul mathématique de l'ACM ---
# On calcule les proportions et les écarts à l'indépendance
N = Z.sum()
P = Z / N
r = P.sum(axis=1)
c = P.sum(axis=0)
S = (P - np.outer(r, c)) / np.sqrt(np.outer(r, c))

# Décomposition mathématique pour trouver les axes principaux
U, sv, Vt = svd(S, full_matrices=False)
lam = sv**2

# Correction pour avoir des pourcentages plus fiables (méthode Benzécri)
mask     = lam > (1/K)**2
sv_f     = sv[mask]
U_f      = U[:, mask]
Vt_f     = Vt[mask, :]
lam_f    = lam[mask]
lam_corr = ((K / (K - 1))**2) * (sv_f - 1/K)**2
total    = lam_corr.sum()
ev       = lam_corr / total * 100
cumev    = np.cumsum(ev)
n_dim    = min(5, len(sv_f))

# Coordonnées des catégories dans l'espace factoriel
G = (Vt_f.T * sv_f[None, :]) * (1.0 / np.sqrt(c))[:, None]

# Noms courts pour les graphiques
short_names = [fn.split('_', 1)[1] if '_' in fn else fn for fn in feature_names]
var_names   = [fn.split('_')[0] for fn in feature_names]

print('ACM calculée avec succès !')
print(f'Nombre de dimensions retenues : {n_dim}')

## Étape 4 — Combien d'information capture chaque axe ?

L'ACM crée plusieurs **axes** (Dim1, Dim2, Dim3...). Chaque axe capture une partie de l'information contenue dans les données.

Le tableau ci-dessous indique **quel pourcentage de l'information** est résumé par chaque axe.

**Comment lire ce tableau ?**
- *% Variance* : la part d'information capturée par cet axe seul
- *% Cumulé* : la part totale capturée en combinant tous les axes jusqu'à celui-là

> En général, on s'intéresse surtout aux **deux premiers axes** (Dim1 et Dim2), car ce sont eux qu'on visualise sur le graphique principal.

In [ ]:
# Tableau des valeurs propres — simplifié
print('='*52)
print('  QUELLE INFORMATION CAPTURE CHAQUE AXE ?')
print('='*52)
print(f'{"Axe":<8}{"% Information":>15}{"% Cumulé":>12}')
print('-'*37)
for i in range(n_dim):
    barre = '#' * int(ev[i] / 2)  # petite barre visuelle
    print(f'Dim {i+1:<4} {ev[i]:>8.1f}%     {cumev[i]:>7.1f}%   {barre}')
print()
print(f'Les 2 premiers axes résument {cumev[1]:.0f}% de l information totale.')

## Étape 5 — Quelles catégories structurent chaque axe ?

Chaque axe est 'construit' par certaines catégories plus que d'autres.

La **contribution** indique l'importance d'une catégorie pour un axe donné.
La somme des contributions sur un axe est toujours **100%**.

**Règle simple :** une catégorie est importante pour un axe si sa contribution est **supérieure à la contribution moyenne** (100 / nombre de catégories).

On affiche ici les **3 catégories les plus importantes** pour chacun des 5 axes.

In [ ]:
# Calcul des contributions
G_contrib = np.zeros((Q, n_dim))
for j in range(n_dim):
    G_contrib[:, j] = c * G[:, j]**2 / lam_f[j] * 100

contrib_df = pd.DataFrame(
    G_contrib[:, :n_dim],
    index=short_names,
    columns=[f'Dim{i+1}' for i in range(n_dim)]
)
contrib_df.insert(0, 'Variable', var_names)

seuil_moyen = round(100 / Q, 1)
print(f'Contribution moyenne (seuil) : {seuil_moyen}%')
print(f'(une valeur > {seuil_moyen}% signifie que la catégorie est importante pour cet axe)')
print()

for i in range(n_dim):
    col = f'Dim{i+1}'
    top3 = contrib_df.nlargest(3, col)
    print(f'Axe {i+1} ({ev[i]:.1f}% de variance) — catégories les plus importantes :')
    for cat, row in top3.iterrows():
        print(f'   {cat:<22} ({row["Variable"]:<12})  {row[col]:.1f}%')
    print()

In [ ]:
# Sauvegarde du tableau complet en CSV
contrib_df.to_csv('contributions_acm.csv')
print('Tableau sauvegardé dans contributions_acm.csv')
print()
print('Tableau complet des contributions (%)')
contrib_df.round(1)

## Étape 6 — Visualisation

On crée maintenant **trois graphiques** pour mieux comprendre les résultats.

### Graphique 1 — Le graphique en barres (Scree plot)
Il montre combien d'information capture chaque axe. Les barres les plus hautes sont les axes les plus importants.

### Graphique 2 — Le plan factoriel (le graphique principal)
C'est la **carte des catégories**. Chaque point représente une catégorie (ex. 'Europa', 'CryoSleep=1', 'CabinDeck B'...). Les couleurs distinguent les variables.

**Comment lire ce graphique ?**
- Des points **proches** = ces catégories vont souvent ensemble
- Des points **éloignés du centre** = catégories rares ou très caractéristiques
- Des points **de part et d'autre du centre** = catégories opposées dans les données

### Graphique 3 — La carte de chaleur des contributions
Elle montre visuellement quelles catégories construisent chaque axe. Plus la couleur est foncée, plus la contribution est forte.

In [ ]:
# Palette de couleurs par variable
colors_var = {
    'HomePlanet':  '#e74c3c',
    'CryoSleep':   '#2ecc71',
    'Destination': '#3498db',
    'VIP':         '#9b59b6',
    'Transported': '#f39c12',
    'CabinDeck':   '#1abc9c',
    'CabinSide':   '#e67e22'
}

# ── Création de la figure ─────────────────────────────────────────
fig = plt.figure(figsize=(20, 22), facecolor='#0f0f1a')
fig.suptitle('ACM — Titanic Spaceship',
             fontsize=18, fontweight='bold', color='white', y=0.98)

gs = fig.add_gridspec(3, 2, hspace=0.5, wspace=0.35,
                      left=0.08, right=0.95, top=0.94, bottom=0.04)

def style_ax(ax, titre):
    ax.set_facecolor('#1a1a2e')
    ax.tick_params(colors='white', labelsize=9)
    ax.set_title(titre, color='white', fontsize=12, fontweight='bold', pad=10)
    for c_ in ['bottom','left']: ax.spines[c_].set_color('#444')
    for c_ in ['top','right']:   ax.spines[c_].set_visible(False)
    ax.xaxis.label.set_color('white')
    ax.yaxis.label.set_color('white')

# ── Graphique 1 : Scree plot ──────────────────────────────────────
ax1 = fig.add_subplot(gs[0, 0])
style_ax(ax1, 'Graphique 1 — Importance de chaque axe')
barres = ax1.bar(range(1, n_dim+1), ev[:n_dim],
                 color=['#e74c3c' if i < 2 else '#3498db' for i in range(n_dim)],
                 edgecolor='white', linewidth=0.5, alpha=0.9)
ax1_bis = ax1.twinx()
ax1_bis.plot(range(1, n_dim+1), cumev[:n_dim], 'yo-', linewidth=2, markersize=7, label='% cumulé')
ax1_bis.set_ylabel('% cumulé', color='yellow', fontsize=9)
ax1_bis.tick_params(colors='yellow', labelsize=9)
ax1_bis.yaxis.label.set_color('yellow')
ax1_bis.spines['right'].set_color('#444')
ax1_bis.spines['top'].set_visible(False)
ax1.set_xlabel('Axe (Dimension)')
ax1.set_ylabel('% variance expliquée')
for barre, pct in zip(barres, ev[:n_dim]):
    ax1.text(barre.get_x() + barre.get_width()/2, barre.get_height() + 0.2,
             f'{pct:.1f}%', ha='center', va='bottom', color='white', fontsize=9, fontweight='bold')

# ── Tableau simplifié des axes ────────────────────────────────────
ax2 = fig.add_subplot(gs[0, 1])
ax2.set_facecolor('#1a1a2e')
ax2.axis('off')
ax2.set_title('Tableau résumé des axes', color='white', fontsize=12, fontweight='bold', pad=10)
entetes = ['Axe', '% Info.', '% Cumulé', 'Catégories principales']
lignes  = []
for i in range(n_dim):
    col = f'Dim{i+1}'
    top2 = contrib_df.nlargest(2, col).index.tolist()
    lignes.append([f'Dim {i+1}', f'{ev[i]:.1f}%', f'{cumev[i]:.1f}%', ', '.join(top2)])
tbl = ax2.table(cellText=lignes, colLabels=entetes, loc='center', cellLoc='center')
tbl.auto_set_font_size(False)
tbl.set_fontsize(9)
for (row, col_), cell in tbl.get_celld().items():
    cell.set_facecolor('#2a2a4a' if row == 0 else ('#222240' if row%2==0 else '#1a1a2e'))
    cell.set_text_props(color='#f39c12' if row==0 else 'white',
                        fontweight='bold' if row==0 else 'normal')
    cell.set_edgecolor('#444')
tbl.scale(1, 2.0)

# ── Graphique 2 : Plan factoriel (carte des catégories) ───────────
ax3 = fig.add_subplot(gs[1, :])
style_ax(ax3, f'Graphique 2 — Carte des catégories (Dim1 = {ev[0]:.0f}%  |  Dim2 = {ev[1]:.0f}%)')
for i, (nom, var) in enumerate(zip(short_names, var_names)):
    couleur = colors_var.get(var, 'white')
    x, y = G[i, 0], G[i, 1]
    ax3.scatter(x, y, color=couleur, s=140, zorder=5, edgecolors='white', linewidth=0.5)
    ax3.annotate(nom, (x, y), textcoords='offset points', xytext=(7, 4),
                 fontsize=9, color=couleur, fontweight='bold')
ax3.axhline(0, color='#555', linewidth=1, linestyle='--')
ax3.axvline(0, color='#555', linewidth=1, linestyle='--')
ax3.set_xlabel(f'Axe 1 — {ev[0]:.1f}% de l information', fontsize=10)
ax3.set_ylabel(f'Axe 2 — {ev[1]:.1f}% de l information', fontsize=10)
patches = [mpatches.Patch(color=colors_var[v], label=v) for v in cat_vars]
ax3.legend(handles=patches, loc='upper right', fontsize=9,
           facecolor='#1a1a2e', edgecolor='#555', labelcolor='white',
           title='Variables', title_fontsize=9)

# ── Graphique 3 : Carte de chaleur des contributions ─────────────
ax4 = fig.add_subplot(gs[2, :])
style_ax(ax4, 'Graphique 3 — Quelles catégories construisent chaque axe ? (plus c est foncé, plus c est important)')
mat = contrib_df.drop('Variable', axis=1).values
im  = ax4.imshow(mat.T, aspect='auto', cmap='YlOrRd', vmin=0, vmax=mat.max())
ax4.set_yticks(range(n_dim))
ax4.set_yticklabels([f'Axe {i+1}' for i in range(n_dim)], color='white', fontsize=9)
ax4.set_xticks(range(len(short_names)))
ax4.set_xticklabels(short_names, rotation=45, ha='right', color='white', fontsize=8)
for i in range(n_dim):
    for j in range(len(short_names)):
        val = mat[j, i]
        if val > 5:
            ax4.text(j, i, f'{val:.0f}%', ha='center', va='center', fontsize=7,
                     color='black' if val > 25 else 'white', fontweight='bold')
cb = plt.colorbar(im, ax=ax4, shrink=0.8, pad=0.01)
cb.ax.tick_params(labelcolor='white', labelsize=8)
cb.set_label('Contribution (%)', color='white', fontsize=9)

plt.savefig('acm_titanic.png', dpi=150, bbox_inches='tight', facecolor='#0f0f1a')
plt.show()
print('Graphique sauvegardé : acm_titanic.png')

## Étape 7 — Comment lire les résultats ?

### Le scree plot (Graphique 1)
- **Dim1** capture environ **20%** de l'information — c'est le plus important
- **Dim2** en capture environ **14%**
- Ensemble, les 2 premiers axes résument environ **34%** de toute l'information

*Ce chiffre peut sembler faible, mais c'est normal en ACM avec beaucoup de variables — 34% sur un seul graphique est déjà très utile !*

---

### La carte des catégories (Graphique 2)

En regardant le plan factoriel, on peut identifier des **profils typiques** :

**Profil 1 — Passager de luxe (côté gauche du graphique) :**
Les catégories *Europa*, *CabinDeck B*, *CabinDeck C* sont proches — les passagers venus d'Europa voyagent souvent dans les ponts haut de gamme.

**Profil 2 — Passager ordinaire (côté droit) :**
Les catégories *Earth*, *CabinDeck F*, *CabinDeck G* apparaissent ensemble — les passagers terriens sont davantage dans les ponts économiques.

**Profil 3 — Voyageurs vers Mars :**
La catégorie *Mars* est bien séparée sur l'axe 2, ce qui montre que les passagers allant vers Mars ont un profil distinct.

---

### La carte de chaleur (Graphique 3)

Elle confirme quelles catégories sont les plus 'responsables' de chaque axe :
- **Axe 1** : surtout *Europa*, *CabinDeck B/C*, *Earth*
- **Axe 2** : surtout *CabinDeck G*, *Mars*, *CryoSleep*
- **Axe 3** : surtout *Mars* et *Transported*

---

### En résumé

> L'ACM révèle que les passagers se regroupent principalement selon leur **planète d'origine** et leur **pont de cabine**. Ces deux variables structurent fortement les données et permettent de distinguer différents profils de voyageurs à bord du Titanic Spaceship.